Import Libraries 

In [ ]:
import pandas as pd
import numpy as np
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import datasets

In [ ]:
!pip install lightgbm
!pip install catboost
!pip install lightgbm
!pip install sklearn
from sklearn import metrics
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Load Dataset

In [ ]:
df_train = pd.read_csv('/content/Test.csv')
df_test = pd.read_csv('/content/Train.csv')

In [ ]:
'concat train and test dataset'
df=pd.concat([df_train,df_test])

Data Pre-processing

In [ ]:
"Label Encode categorical variable country"
country=df['country'].unique()
num=range(df['country'].nunique())
country_dict={}
for a,b in zip(country, num):
    df.replace(a,b, inplace=True)
    country_dict[a]=b



In [ ]:
"Label Encode categorical variable region"
region=df['region'].unique()
num=range(df['region'].nunique())
region_dict={}
for a,b in zip(region, num):
    df.replace(a,b, inplace=True)
    region_dict[a]=b

In [ ]:
'column Q7 has about 80% missing values, hence dropped'
df.drop('Q7', inplace=True, axis=1)

In [ ]:
df_train.shape

(44943, 35)

In [ ]:
df_test.shape


(104867, 36)

In [ ]:
'Split train and test Dataset'
df_train=df.iloc[44943:]
df_test=df.iloc[:44943]

In [ ]:
df_X=df_train.drop(['target', 'ID'], axis=1)
df_Y=df_train[['target']]

In [ ]:
'Fill missing values with mean'
for i in ['region','age', 'Q2','Q3','Q4','Q5','Q8','Q9','Q19', 'Q20','Q23','Q26','Q27','Q28', 'owns_mobile']:
      df_X.replace(np.nan, df_X[i].mean(),inplace=True)
      df_test.replace(np.nan, df_test[i].mean(),inplace=True)

      

/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


Modelling Preparation

In [ ]:

catboost=CatBoostClassifier()
lightgbm=LGBMClassifier()

In [ ]:

vc=VotingClassifier(estimators=[('lg',lightgbm), ('cat', catboost)],
voting='soft', weights=[0.1, 0.35] )

Modelling and Prediction

In [ ]:
ID=df_test[['ID']]

In [ ]:
df_test.drop(['ID','target'], axis=1, inplace=True)

/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
df_Y

,target
0,0.0
1,1.0
2,0.0
3,1.0
4,1.0
...,...
104862,0.0
104863,1.0
104864,0.0
104865,1.0


In [ ]:
#stratified kfolding with voting classifier


predictions=list()
kf=StratifiedKFold(n_splits=25)
for Train, Test in kf.split(df_X , df_Y):

    x_train,x_test,y_train,y_test=df_X.iloc[Train], df_X.iloc[Test], df_Y.iloc[Train], df_Y.iloc[Test]
    vc.fit(x_train, y_train)
    prediction=vc.predict_proba(df_test)[:,1]
    predictions.append(prediction)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Learning rate set to 0.07382
0:	learn: 0.6777525	total: 29.2ms	remaining: 29.1s
1:	learn: 0.6636426	total: 59.7ms	remaining: 29.8s
2:	learn: 0.6519463	total: 89.9ms	remaining: 29.9s
3:	learn: 0.6420081	total: 121ms	remaining: 30.2s
4:	learn: 0.6336289	total: 153ms	remaining: 30.4s
5:	learn: 0.6266783	total: 182ms	remaining: 30.2s
6:	learn: 0.6203756	total: 216ms	remaining: 30.6s
7:	learn: 0.6149928	total: 246ms	remaining: 30.5s
8:	learn: 0.6102638	total: 276ms	remaining: 30.3s
9:	learn: 0.6061446	total: 307ms	remaining: 30.4s
10:	learn: 0.6023714	total: 337ms	remaining: 30.3s
11:	learn: 0.5992011	total: 368ms	remaining: 30.3s
12:	learn: 0.5963844	total: 399ms	remaining: 30.3s
13:	learn: 0.5939103	total: 434ms	remaining: 30.6s
14:	learn: 0.5916875	total: 466ms	remaining: 30.6s
15:	learn: 0.5896048	total: 501ms	remaining: 30.8s
16:	learn: 0.5874168	total: 531ms	remaining: 30.7s
17:	learn: 0.5859592	total: 562ms	remaining: 30.7s
18:	learn: 0.5840622	total: 603ms	remaining: 31.1s
19:	learn

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Learning rate set to 0.07382
0:	learn: 0.6778104	total: 30.6ms	remaining: 30.6s
1:	learn: 0.6643606	total: 62.8ms	remaining: 31.3s
2:	learn: 0.6524593	total: 91.8ms	remaining: 30.5s
3:	learn: 0.6428294	total: 123ms	remaining: 30.7s
4:	learn: 0.6344935	total: 155ms	remaining: 30.8s
5:	learn: 0.6273335	total: 192ms	remaining: 31.7s
6:	learn: 0.6210719	total: 222ms	remaining: 31.5s
7:	learn: 0.6155732	total: 252ms	remaining: 31.3s
8:	learn: 0.6108650	total: 283ms	remaining: 31.1s
9:	learn: 0.6066912	total: 315ms	remaining: 31.2s
10:	learn: 0.6028757	total: 345ms	remaining: 31.1s
11:	learn: 0.5996967	total: 379ms	remaining: 31.2s
12:	learn: 0.5965184	total: 414ms	remaining: 31.4s
13:	learn: 0.5938310	total: 444ms	remaining: 31.3s
14:	learn: 0.5916688	total: 477ms	remaining: 31.3s
15:	learn: 0.5897122	total: 507ms	remaining: 31.2s
16:	learn: 0.5875633	total: 538ms	remaining: 31.1s
17:	learn: 0.5855983	total: 569ms	remaining: 31s
18:	learn: 0.5836212	total: 603ms	remaining: 31.1s
19:	learn: 

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


...

Learning rate set to 0.073821
0:	learn: 0.6770934	total: 33.2ms	remaining: 33.2s
1:	learn: 0.6635632	total: 67.3ms	remaining: 33.6s
2:	learn: 0.6520413	total: 99.5ms	remaining: 33.1s
3:	learn: 0.6422211	total: 145ms	remaining: 36.1s
4:	learn: 0.6337401	total: 178ms	remaining: 35.5s
5:	learn: 0.6268452	total: 234ms	remaining: 38.7s
6:	learn: 0.6206934	total: 268ms	remaining: 38s
7:	learn: 0.6152185	total: 298ms	remaining: 37s
8:	learn: 0.6105680	total: 328ms	remaining: 36.1s
9:	learn: 0.6063886	total: 360ms	remaining: 35.6s
10:	learn: 0.6026668	total: 392ms	remaining: 35.2s
11:	learn: 0.5995416	total: 457ms	remaining: 37.6s
12:	learn: 0.5966840	total: 509ms	remaining: 38.7s
13:	learn: 0.5938720	total: 542ms	remaining: 38.1s
14:	learn: 0.5913559	total: 587ms	remaining: 38.6s
15:	learn: 0.5893227	total: 621ms	remaining: 38.2s
16:	learn: 0.5871740	total: 654ms	remaining: 37.8s
17:	learn: 0.5853973	total: 745ms	remaining: 40.7s
18:	learn: 0.5839192	total: 839ms	remaining: 43.3s
19:	learn: 0

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Learning rate set to 0.073821
0:	learn: 0.6771520	total: 28.3ms	remaining: 28.2s
1:	learn: 0.6636075	total: 58.5ms	remaining: 29.2s
2:	learn: 0.6520283	total: 88.8ms	remaining: 29.5s
3:	learn: 0.6419581	total: 120ms	remaining: 29.9s
4:	learn: 0.6338569	total: 150ms	remaining: 29.9s
5:	learn: 0.6269566	total: 179ms	remaining: 29.7s
6:	learn: 0.6205560	total: 233ms	remaining: 33s
7:	learn: 0.6149270	total: 263ms	remaining: 32.6s
8:	learn: 0.6102389	total: 296ms	remaining: 32.6s
9:	learn: 0.6061820	total: 328ms	remaining: 32.4s
10:	learn: 0.6023975	total: 356ms	remaining: 32s
11:	learn: 0.5991907	total: 387ms	remaining: 31.8s
12:	learn: 0.5963702	total: 417ms	remaining: 31.6s
13:	learn: 0.5939913	total: 467ms	remaining: 32.9s
14:	learn: 0.5916884	total: 500ms	remaining: 32.8s
15:	learn: 0.5891551	total: 528ms	remaining: 32.5s
16:	learn: 0.5870162	total: 572ms	remaining: 33.1s
17:	learn: 0.5851916	total: 602ms	remaining: 32.8s
18:	learn: 0.5838036	total: 632ms	remaining: 32.7s
19:	learn: 0

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Learning rate set to 0.073821
0:	learn: 0.6770369	total: 44.6ms	remaining: 44.6s
1:	learn: 0.6635481	total: 74.9ms	remaining: 37.4s
2:	learn: 0.6518885	total: 105ms	remaining: 35s
3:	learn: 0.6421344	total: 137ms	remaining: 34.2s
4:	learn: 0.6337102	total: 168ms	remaining: 33.5s
5:	learn: 0.6267377	total: 217ms	remaining: 36s
6:	learn: 0.6203645	total: 257ms	remaining: 36.5s
7:	learn: 0.6148725	total: 288ms	remaining: 35.7s
8:	learn: 0.6100681	total: 318ms	remaining: 35s
9:	learn: 0.6056179	total: 346ms	remaining: 34.3s
10:	learn: 0.6021322	total: 376ms	remaining: 33.8s
11:	learn: 0.5984639	total: 410ms	remaining: 33.7s
12:	learn: 0.5956625	total: 461ms	remaining: 35s
13:	learn: 0.5929028	total: 491ms	remaining: 34.6s
14:	learn: 0.5904669	total: 519ms	remaining: 34.1s
15:	learn: 0.5884507	total: 553ms	remaining: 34s
16:	learn: 0.5867064	total: 583ms	remaining: 33.7s
17:	learn: 0.5849931	total: 614ms	remaining: 33.5s
18:	learn: 0.5829482	total: 647ms	remaining: 33.4s
19:	learn: 0.581110

In [ ]:
sub=np.mean(predictions, axis=0)
target=pd.DataFrame(sub)

In [ ]:
dfs = pd.concat([ID, target], axis=1)
dfs = dfs.rename({'ID':'ID', 0:'target'}, axis=1)


In [ ]:
dfs.to_csv('Dont Overfit 2.csv', index = False)


In [ ]:
from google.colab import files
files.download('Dont Overfit 2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>